In [ ]:
! pip install moviepy

In [1]:
import pandas as pd
import numpy as np

In [14]:
from moviepy.editor import VideoFileClip
from moviepy.editor import VideoFileClip, concatenate_videoclips, ColorClip

MAX_FRAMES = 80

def save_subclip_from_frames(video_path, start_frame, end_frame, output_path,fps):
    # Load the original video
    clip = VideoFileClip(video_path)

    # Get the frame rate of the video to calculate start and end times in seconds
    # fps = clip.fps
    start_time = start_frame / fps
    end_time = end_frame / fps
    # Extract the subclip
    subclip = clip.subclip(start_time, end_time)
    # Calculate the current number of frames
    current_frames = int((end_time - start_time) * clip.fps)

    # Check if padding is needed
    if current_frames < MAX_FRAMES:
        # Calculate the number of frames needed for padding
        padding_frames = MAX_FRAMES - current_frames

        # Calculate the duration of each black frame and the total padding duration
        padding_duration_per_frame = 1 / clip.fps
        total_padding_duration = padding_frames * padding_duration_per_frame

        # Create a black clip of needed padding duration
        black_clip = ColorClip(size=subclip.size, color=(0,0,0), duration=total_padding_duration).set_fps(fps)

        # Concatenate subclip with black_clip
        final_clip = concatenate_videoclips([subclip, black_clip], method="compose")
        final_clip.write_videofile(output_path,codec="libx264" ,audio=False)

    else:
        if current_frames > MAX_FRAMES:
          print('Drop!!!')
        else:
          final_clip = subclip
          final_clip.write_videofile(output_path,codec="libx264" ,audio=False)



    print('Hey')

    # Close the clips to free resources
    clip.close()
    subclip.close()
    if 'black_clip' in locals():
        black_clip.close()
    if 'final_clip' in locals():
        final_clip.close()



    # Save the subclip as a new video file in AVI format without specifying an audio codec

    # Close the clips to release resources
    clip.close()
    subclip.close()

def split_videos(video_files, train_ratio=0.7, validation_ratio=0.2, test_ratio=0.1):
    """Shuffle and split the list of video files into training, validation, and test sets."""
    if not np.isclose(train_ratio + validation_ratio + test_ratio, 1.0):
        raise ValueError("Ratios must sum to 1.0")

    np.random.seed(42)  # For reproducibility
    np.random.shuffle(video_files)

    n = len(video_files)
    end_train = int(n * train_ratio)
    end_validation = end_train + int(n * validation_ratio)

    return video_files[:end_train], video_files[end_train:end_validation], video_files[end_validation:]


In [3]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/YData/Project/VIZ.AI project/EchoNet-Dynamic/Videos/'
filename = '0X140D4A2F475712FA.avi'
video_path = folder_path+filename
# save_subclip_from_frames(video_path, start_frame, end_frame, output_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
volumeTraces = pd.read_csv('/content/drive/MyDrive/YData/Project/VIZ.AI project/EchoNet-Dynamic/VolumeTracings.csv')


In [4]:

# Specify output folders
folders = {
    'train': '/content/drive/MyDrive/YData/Project/Code/3D_CNN_Data/train',
    'val': '/content/drive/MyDrive/YData/Project/Code/3D_CNN_Data/val',
    'test': '/content/drive/MyDrive/YData/Project/Code/3D_CNN_Data/test'
}

In [13]:

frmaesData = pd.read_csv('/content/drive/MyDrive/YData/Project/VIZ.AI project/EchoNet-Dynamic/FileList.csv')
volumeTraces = pd.read_csv('/content/drive/MyDrive/YData/Project/VIZ.AI project/EchoNet-Dynamic/VolumeTracings.csv')
heartBeat_Frames  = volumeTraces.groupby('FileName')['Frame'].agg(['min','max']).reset_index()
frmaesData['FileName'] =frmaesData['FileName'].apply(lambda x: x+'.avi')
train_files, validation_files, test_files = split_videos(heartBeat_Frames['FileName'].tolist())
heartBeat_Frames['set'] = heartBeat_Frames['FileName'].apply(lambda x: 'train' if x in train_files else ('val' if x in validation_files else 'test'))

result = pd.merge(heartBeat_Frames, frmaesData, how="left", on=["FileName"])
display(result)

,FileName,min,max,set,EF,ESV,EDV,FrameHeight,FrameWidth,FPS,NumberOfFrames,Split
0,0X100009310A3BD7FC.avi,46,61,train,78.498406,14.881368,69.210534,112.0,112.0,50.0,174.0,VAL
1,0X1002E8FBACD08477.avi,3,18,val,59.101988,40.383876,98.742884,112.0,112.0,50.0,215.0,TRAIN
2,0X1005D03EED19C65B.avi,24,35,train,62.363798,14.267784,37.909734,112.0,112.0,50.0,104.0,TRAIN
3,0X10075961BC11C88E.avi,91,108,train,54.545097,33.143084,72.914210,112.0,112.0,55.0,122.0,TRAIN
4,0X10094BA0A028EAC3.avi,137,156,test,24.887742,127.581945,169.855024,112.0,112.0,52.0,207.0,VAL
...,...,...,...,...,...,...,...,...,...,...,...,...
10020,0XFDFBA5702E94ABF.avi,100,117,train,53.648451,40.224359,86.781046,112.0,112.0,62.0,192.0,TRAIN
10021,0XFDFD17B7CCFE5AF.avi,66,85,train,63.338180,28.333805,77.284229,112.0,112.0,50.0,268.0,TRAIN
10022,0XFE6E32991136338.avi,31,45,val,32.783248,113.063526,168.207364,112.0,112.0,43.0,120.0,TEST
10023,0XFE83FF3D3B13C3A.avi,49,67,train,64.084988,13.714901,38.187097,112.0,112.0,50.0,192.0,VAL


In [ ]:
failed_list = []
for index, row in result.iterrows():
  filename = row['FileName']
  start_frame = row['min']  # Starting frame of the new clip
  end_frame = row['max']    # Ending frame of the new clip
  fps = row['FPS']
  output_path = folders[row['set']] + '/' + filename
  try:
    save_subclip_from_frames(video_path, start_frame, end_frame, output_path,fps)
  except Exception as e:
    print(e)
    print(filename)

    failed_list.append(filename)
    continue
print(f'Failed in over all {len(failed_list)} files\n {failed_list}')

In [ ]:
failed_list

NameError: name 'failed_list' is not defined